# Thực hiện 9 loại kiểm định cho bảng chéo

**BS. Lê Ngọc Khả Nhi**

Trong bài này, Nhi tạo ra 1 công cụ phân tích bảng chéo, cho phép thực hiện đồng thời 6 loại kiểm định theo phương pháp phổ quát là Power divergence statistic và goodness of fit test, do 2 tác giả Noel Cressie và Timothy R. C. Read lập ra vào năm 1984.

Phương pháp Power divergence của Cressie và Read có tham số $\lambda$, gồm các giá trị có thể:

+ $\lambda$ = 1 tương đương với kiểm định Chi squared cổ điển cùa Pearson

+ $\lambda$ = 0 tương đương với G-test (log-likelihood ratio test)

+ $\lambda$ = -0.5 tương đương với trị số thống kê của Freeman và Tukey

+ $\lambda$ = -1 tương đương với kiểm định log-likelihood ratio hiệu chỉnh

+ $\lambda$ = -2 tương đương với trị số thống kê của Neyman

+ $\lambda$ = 2/3 là giá trị đề xuất bởi Cressie và Read năm 1984

Ngoài ra, nếu là bảng chéo 2x2, ta có thể thực hiện 3 loại kiểm định chính xác (Exact tests) bao gồm :

+ Kiểm định Fisher (Fisher exact test, năm 1922)

+ Kiểm định Boschloo (1970) 

+ Kiểm định Barnard (1947)

In [1]:
warning_status = "ignore"
import warnings
warnings.filterwarnings(warning_status)
with warnings.catch_warnings():
    warnings.filterwarnings(warning_status, category=DeprecationWarning)

import numpy as np
import pandas as pd

from scipy.stats.contingency import expected_freq
from scipy.stats import power_divergence, boschloo_exact, fisher_exact, barnard_exact

from dataclasses import dataclass, field

def check_variable(data: pd.DataFrame, x: str, y:str) -> bool:

    return all(i in data.columns for i in [x, y])

@dataclass
class Contingency_test:

    data: pd.DataFrame = field(init=True)
    x: str = field(init=True)
    y: str = field(init=True)

    def __post_init__(self):
        if not check_variable(self.data, self.x, self.y):
            raise ValueError("x or y not in data columns")

        self.observed = pd.crosstab(self.data[self.x], self.data[self.y])

        self.expected = pd.DataFrame(
            expected_freq(self.observed),
            index=self.observed.index,
            columns=self.observed.columns,
        )

        for xtab, name in zip([self.observed, self.expected], ["Quan sát", "Giả định"]):

            if (xtab < 5).any(axis=None):
                print(f"Lưu ý: có giá trị nhỏ hơn 5 trong bảng {name}")

        print("Bảng phân bố tần suất")
        print("-" * 20)
        print(self.observed)

    def power_divergence(self, correction: bool = False) -> pd.DataFrame:
        """
        Cressie-Read power divergence statistic and goodness of fit test.
        """
        dof = float(
            self.expected.size - sum(self.expected.shape) + self.expected.ndim - 1
        )

        n = self.observed.values.sum()

        if dof == 1 and correction:
            self.observed = self.observed + 0.5 * np.sign(self.expected - self.observed)

        ddof = self.observed.size - 1 - dof

        methods = [
            "Pearson Chi2",
            "Cressie-Read",
            "G-test",
            "Freeman-Tukey",
            "mod-log-LR",
            "Neyman",
        ]

        stats = []

        for met, lambda_ in zip(methods, [1.0, 2 / 3, 0.0, -1 / 2, -1.0, -2.0]):

            if dof == 0:
                chi2, p, cramer = 0.0, 1.0, "NA"
            else:
                chi2, p = power_divergence(
                    self.observed, self.expected, ddof=ddof, axis=None, lambda_=lambda_
                )

                cramer = np.sqrt(chi2 / (n * (min(self.expected.shape) - 1)))

            stats.append(
                {
                    "Phương pháp": met,
                    "df": dof,
                    "\u03C7" + "\u00b2": chi2,
                    "Cramer's V": cramer,
                    "Giá trị p": p,
                    "Phủ định H0": "Có" if p < 0.05 else "Không",
                }
            )

        out = pd.DataFrame(stats)

        return out.style.hide_index()

    def exact_test(self) -> pd.DataFrame:
        """
        Exact tests for 2x2 cross-table
        """
        # verify that the table is 2x2
        if self.observed.shape != (2, 2):
            raise ValueError("Chỉ áp dụng cho bảng chéo 2x2")

        methods = ["Boschloo", "Fisher", "Barnard"]
        stats = []

        for i, met in enumerate(methods):

            if i == 0:
                res = boschloo_exact(self.observed)
                stat, p = res.statistic, res.pvalue
            elif i == 1:
                stat, p = fisher_exact(self.observed)
            elif i == 2:
                res = barnard_exact(self.observed)
                stat, p = res.statistic, res.pvalue

            stats.append(
                {
                    "Phương pháp": met,
                    "Trị số tk": stat,
                    "Giá trị p": p,
                    "Phủ định H0": "Có" if p < 0.05 else "Không",
                }
            )

        out = pd.DataFrame(stats)

        return out.style.hide_index()

# Thí dụ minh họa:  Bộ dữ liệu Sinh non và Covid-19

In [2]:
df = pd.read_excel('Covid_pretermbirth.xlsx')

df

,ID,Tuổi,COVID,Tiền sử SN,Tiền sử PT,Tuổi thai,Sinh non,ĐTĐTK
0,1,30-35,1,0,0,260,0,0
1,2,30-35,1,0,0,274,0,0
2,3,30-35,1,1,0,268,0,0
3,4,30-35,1,0,0,277,0,1
4,5,30-35,1,0,0,277,0,0
...,...,...,...,...,...,...,...,...
224,225,35-40,0,0,0,244,1,0
225,226,35-40,0,0,0,240,1,0
226,227,30-35,0,0,0,253,1,0
227,228,30-35,0,0,0,251,1,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          229 non-null    int64 
 1   Tuổi        229 non-null    object
 2   COVID       229 non-null    int64 
 3   Tiền sử SN  229 non-null    int64 
 4   Tiền sử PT  229 non-null    int64 
 5   Tuổi thai   229 non-null    int64 
 6   Sinh non    229 non-null    int64 
 7   ĐTĐTK       229 non-null    int64 
dtypes: int64(7), object(1)
memory usage: 14.4+ KB


In [4]:
df.columns

Index(['ID', 'Tuổi', 'COVID', 'Tiền sử SN', 'Tiền sử PT', 'Tuổi thai',
       'Sinh non', 'ĐTĐTK'],
      dtype='object')

In [5]:
test = Contingency_test(df, 'COVID','Sinh non')

Bảng phân bố tần suất
--------------------
Sinh non   0   1
COVID           
0         95  81
1         32  21


In [6]:
test.power_divergence(correction = True)

Phương pháp,df,χ²,Cramer's V,Giá trị p,Phủ định H0
Pearson Chi2,1.000000,0.441202,0.043894,0.506543,Không
Cressie-Read,1.000000,0.441892,0.043928,0.506211,Không
G-test,1.000000,0.443406,0.044003,0.505483,Không
Freeman-Tukey,1.000000,0.444661,0.044065,0.504881,Không
mod-log-LR,1.000000,0.446018,0.044133,0.504232,Không
Neyman,1.000000,0.449050,0.044282,0.502786,Không


In [7]:
test.exact_test()

Phương pháp,Trị số tk,Giá trị p,Phủ định H0
Boschloo,0.302446,0.516395,Không
Fisher,0.804435,0.528414,Không
Barnard,-0.679029,0.609462,Không


In [8]:
from IPython.display import display

for k in ['COVID', 'Tiền sử SN', 'Tiền sử PT', 'ĐTĐTK']:
    
    test = Contingency_test(df, k,'Sinh non')
    out1 = test.power_divergence()
    display(out1)
    out2 = test.exact_test()
    display(out2)
    print('-'*50)

Bảng phân bố tần suất
--------------------
Sinh non   0   1
COVID           
0         95  81
1         32  21


Phương pháp,df,χ²,Cramer's V,Giá trị p,Phủ định H0
Pearson Chi2,1.000000,0.675448,0.054310,0.411159,Không
Cressie-Read,1.000000,0.676795,0.054364,0.410693,Không
G-test,1.000000,0.679807,0.054485,0.409653,Không
Freeman-Tukey,1.000000,0.682349,0.054587,0.408779,Không
mod-log-LR,1.000000,0.685137,0.054698,0.407824,Không
Neyman,1.000000,0.691471,0.054950,0.405665,Không


Phương pháp,Trị số tk,Giá trị p,Phủ định H0
Boschloo,0.253958,0.434415,Không
Fisher,0.769676,0.434737,Không
Barnard,-0.821856,0.420205,Không


--------------------------------------------------
Lưu ý: có giá trị nhỏ hơn 5 trong bảng Quan sát
Lưu ý: có giá trị nhỏ hơn 5 trong bảng Giả định
Bảng phân bố tần suất
--------------------
Sinh non      0   1
Tiền sử SN         
0           125  95
1             2   7


Phương pháp,df,χ²,Cramer's V,Giá trị p,Phủ định H0
Pearson Chi2,1.000000,4.189359,0.135256,0.040678,Có
Cressie-Read,1.000000,4.192511,0.135307,0.040603,Có
G-test,1.000000,4.310916,0.137204,0.037869,Có
Freeman-Tukey,1.000000,4.510856,0.140350,0.033680,Có
mod-log-LR,1.000000,4.825607,0.145164,0.028040,Có
Neyman,1.000000,5.917844,0.160755,0.014988,Có


...

Phương pháp,Trị số tk,Giá trị p,Phủ định H0
Boschloo,0.122303,0.199034,Không
Fisher,0.652063,0.206827,Không
Barnard,-1.320749,0.202961,Không


--------------------------------------------------
Bảng phân bố tần suất
--------------------
Sinh non    0   1
ĐTĐTK            
0         115  85
1          12  17


Phương pháp,df,χ²,Cramer's V,Giá trị p,Phủ định H0
Pearson Chi2,1.000000,2.664568,0.107869,0.102606,Không
Cressie-Read,1.000000,2.655831,0.107692,0.103171,Không
G-test,1.000000,2.648715,0.107547,0.103634,Không
Freeman-Tukey,1.000000,2.652425,0.107623,0.103392,Không
mod-log-LR,1.000000,2.663943,0.107856,0.102646,Không
Neyman,1.000000,2.710933,0.108803,0.099663,Không


Phương pháp,Trị số tk,Giá trị p,Phủ định H0
Boschloo,0.076360,0.122885,Không
Fisher,1.916667,0.113344,Không
Barnard,1.632350,0.109453,Không


--------------------------------------------------


# Tài liệu tham khảo

Noel Cressie and Timothy R. C. Read. Multinomial goodness‐of‐fit tests. Journal of the Royal Statistical Society: Series B (Methodological), 1984; 46(3), 440-464.

Freeman, M.F. and J.W. Tukey (1950), Transformations related to the angular and the square root, Ann. Math. Statist. 21, 607-611.

Campbell B. Read. Freeman--Tukey chi-squared goodness-of-fit statistics. Statistics & Probability Letters, 1993, vol. 18, issue 4, 271-278.

J. Neyman and E. S. Pearson, On the use and interpretation of certain test criteria for purposes of statistical inference: Part i, Biometrika 20A (1928) 175–240.

Xiangpan Ji, Wenqiang Gu, Xin Qian, Hanyu Wei, Chao Zhang. Combined Neyman-Pearson Chi-square: An Improved Approximation to the Poisson-likelihood Chi-square. Nuclear Instruments and Methods in Physics Research Section A: Accelerators, Spectrometers, Detectors and Associated Equipment 2020; 961:163677

Frank Yates. Contingency Tables Involving Small Numbers and the  𝜒2𝑡𝑒𝑠𝑡 . Supplement to the Journal of the Royal Statistical Society, 1934; 1: 217-235.


Fisher, R. A. (1922). "On the interpretation of χ2 from contingency tables, and the calculation of P". Journal of the Royal Statistical Society. 85 (1): 87–94. doi:10.2307/2340521. JSTOR 2340521.

R.D. Boschloo. “Raised conditional level of significance for the 2 x 2-table when testing the equality of two probabilities”, Statistica Neerlandica, 24(1), 1970

https://en.wikipedia.org/wiki/Fisher%27s_exact_test
https://en.wikipedia.org/wiki/Barnard%27s_test
https://en.wikipedia.org/wiki/Boschloo%27s_test

Barnard, G. A. “Significance Tests for 2x2 Tables”. Biometrika. 34.1/2 (1947): 123-138. DOI:dpgkg3